In [3]:
import torch
import torch.nn.functional as F
from transformers import RobertaTokenizer, RobertaModel

In [4]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")

In [5]:
query = "Write a Python function to compute factorial"
code = """
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
"""